# Clustering trials

## Setup

### Imports

In [ ]:
import json, random, sys
import plotly.express as px
import numpy as np
import pandas as pd
from loguru import logger
from rich import print
import matplotlib.pyplot as plt

import umap
from sklearn.cluster import KMeans, HDBSCAN
from sklearn.metrics import silhouette_score

import torch
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
# check GPU availability
sys.path.append("../")
from src.ai_news_digest.utils import check_gpu_availability, create_run_folder
from src.ai_news_digest.steps.clustering_utils import (
    clustering_pipeline,
    entropy,
    grid_search_umap_projection,
    grid_search_clustering,
)
import src.ai_news_digest.steps.load_news_corpus as lnc
import src.ai_news_digest.steps.load_arxiv_corpus as lac

### Set random seeds

In [ ]:
np.random.seed(123)
random.seed(123)
torch.manual_seed(123)

### Check GPU availability

In [ ]:
device = check_gpu_availability()

### Config

In [ ]:
MODEL_CONFIG_PATH = "../conf/base/cluster_bench_models.yml"
DATA_PATH = "../data/03_primary/arxiv_dict_2023-11-06_00-22-42.json"

MODEL_KWARGS = {"device": device}
ENCODE_KWARGS = {
    "normalize_embeddings": True,
    "batch_size": 16,
    "output_value": "sentence_embedding",
    "convert_to_numpy": True,
    "show_progress_bar": True,
}

In [ ]:
PATH_INFO_DICT = "data/03_primary/arxiv_dict_2023-11-06_00-22-42.json"
MODEL_ID = "BAAI/bge-small-en"

## Detailed Clustering Pipeline

### Load data

In [ ]:
# # load data
# with open(DATA_PATH, "r") as f:
#     info_dict = json.load(f)["results"]
# logger.info(f"Successfully loaded prepared data from : {DATA_PATH}")

# # retrieve abstracts, titles, dates & paper IDs
# df_data = pd.DataFrame(info_dict).transpose()

# # display
# df_data.head()

In [ ]:
# query 
res_gnews = lnc.load_news_gnews(
    keywords="semi conductors",
    language="en",
    period="90d",
    start_date=None,
    end_date=None,
    exclude_websites=[],
    max_results=300,
    override_content=True,
    use_logs=False,
    standardize=True,
)

# store in dataframe
df_data = pd.DataFrame({d["url"]: d for d in res_gnews}).transpose()

In [ ]:
df_data.dropna(subset=["content"], inplace=True)
df_data.head()


### Embed Data

In [ ]:
# load model
hf = HuggingFaceEmbeddings(
    model_name=MODEL_ID,
    model_kwargs={"device": device},
    encode_kwargs=ENCODE_KWARGS,
)

In [ ]:
# compute embeddings
# embeddings = np.array(hf.embed_documents(df_data["abstract"]))
embeddings = np.array(hf.embed_documents(df_data["content"]))

# store embeddings in a dataframe
df_embed = pd.DataFrame(
    data=embeddings, 
    columns=[f"embed_{i}" for i in range(embeddings.shape[1])],
    index=df_data.index
)

# display
df_embed.head()

### Dimensionality reduction

In [ ]:
# set umap params
umap_kwargs = {
    "n_neighbors": 5,
    "min_dist": 0.001,
    "n_components": 2,
    "metric": "cosine",
}

# instanciate umap projector
reducer = umap.UMAP(random_state=123, **umap_kwargs)

# project data
umap_proj = reducer.fit_transform(df_embed)

# normalize umap coords
umap_proj = (umap_proj - umap_proj.min(axis=0)) / (umap_proj.max(axis=0) - umap_proj.min(axis=0))

# store in a dataframe with metadata
df_umap = pd.DataFrame(columns=[f"umap_{i}" for i in range(umap_proj.shape[1])], data=umap_proj)
df_umap = pd.concat((df_umap, df_data.reset_index(names=["ID"])), axis=1)

# display
df_umap.head()

### Clustering

In [ ]:
X_cluster = df_umap[[col for col in df_umap.columns if "umap" in col]]

clustering = HDBSCAN(
    min_cluster_size=10, 
    min_samples=3, 
    max_cluster_size=None, 
    cluster_selection_epsilon=0.05
)
clustering.fit(X_cluster)

In [ ]:
# viz
df_umap["cluster"] = [str(elt) for elt in clustering.labels_]
df_umap["noise"] = [int(elt==-1) for elt in clustering.labels_]
fig = px.scatter(
    df_umap,
    x="umap_0",
    y="umap_1",
    hover_data=[
        "title",
        "ID",
    ],
    color="cluster",
    symbol="noise",
    # color_continuous_scale=px.colors.qualitative.D3,
    category_orders={"cluster": list(np.sort(pd.unique(clustering.labels_)).astype(str))},
)
fig.show()

In [ ]:
# Entropy ↓
entropy(X_cluster.values)

In [ ]:
# Silhouette ↑
silhouette_score(X_cluster, df_umap["cluster"])

## Refactored Clustering Pipeline

In [ ]:
clustering_kwargs = {    
    "min_cluster_size": 10, 
    "min_samples": 3, 
    "max_cluster_size": None, 
    "cluster_selection_epsilon": 0.05,
}

umap_kwargs = {
    "n_neighbors": 5,
    "min_dist": 0.1,
    "n_components": 2,
    "metric": "cosine",
}

df_umap, clustering = clustering_pipeline(
    df_embed,
    umap_kwargs,
    clustering_kwargs,
    random_state=123,
    df_data=df_data,
)

X_cluster = df_umap[[c for c in df_umap.columns if "umap_" in c]]

# Silhouette ↑
print(f" Silhouette ↑ = {silhouette_score(X_cluster, df_umap['cluster'])}")

# Entropy ↓
print(f" Entropy ↓ = {entropy(X_cluster.values)}")

# viz
df_umap["cluster"] = [str(elt) for elt in clustering.labels_]
df_umap["noise"] = [int(elt==-1) for elt in clustering.labels_]
fig = px.scatter(
    df_umap,
    x="umap_0",
    y="umap_1",
    hover_data=[
        "title",
        # "ID",
    ],
    color="cluster",
    symbol="noise",
    # color_continuous_scale=px.colors.qualitative.D3,
    category_orders={"cluster": list(np.sort(pd.unique(clustering.labels_)).astype(str))},
)
fig.show()

## Grid search over projection and clustering parameters

#### Optimize UMAP Projection

In [ ]:
# from itertools import product
from tqdm.contrib.itertools import product

clustering_grid = {
    "min_cluster_size": [3, 5, 10, 15], 
    "min_samples": [3, 5, 10], 
    "max_cluster_size": [
        None, 
        15, 
        25, 
        40,
    ], 
    "cluster_selection_epsilon": [
        # 0.0, 
        # 0.05, 
        # 0.0,
        # 0.001,
        # 0.01,
        0.05,
        0.1,    
        0.5,    
    ],
}

umap_grid = {
    "n_neighbors": [
        # 2, 
        4, 
        8, 
        10,
    ],
    "min_dist": [0.001, 0.01, 0.1, 0.5],
    "n_components": [2],
    "metric": ["cosine"],
    "init": ["random"],
}


In [ ]:
best_entropy, best_umap_kwargs, df_umap_search = grid_search_umap_projection(
    umap_grid=umap_grid,
    df_embed=df_embed,
    random_state=123,
    df_data=None,
)

In [ ]:
df_umap_search

In [ ]:
fig = px.scatter_matrix(df_umap_search)
fig.show()

#### Optimize Clustering

In [ ]:
best_silhouette, best_clustering_kwargs, df_clustering_search = grid_search_clustering(
    clustering_grid=clustering_grid,
    df_embed=df_embed,
    umap_kwargs=best_umap_kwargs,
    random_state=123,
    df_data=df_data,    
)

In [ ]:
df_clustering_search.sort_values(by="Silhouette ↑").tail(50)

In [ ]:
df_clustering_search["Silhouette ↑"].sort_values().reset_index(drop=True).plot(figsize=(8,3), grid=True)
plt.show()

In [ ]:
fig = px.scatter_matrix(df_clustering_search, height=800)
fig.show()

## Apply Optimized Pipeline

In [ ]:
df_umap, clustering = clustering_pipeline(
    df_embed,
    best_umap_kwargs,
    best_clustering_kwargs,
    random_state=123,
    df_data=df_data,
)

X_cluster = df_umap[[c for c in df_umap.columns if "umap_" in c]]

# Silhouette ↑
print(f" Silhouette ↑ = {silhouette_score(X_cluster, df_umap['cluster'])}")

# Entropy ↓
print(f" Entropy ↓ = {entropy(X_cluster.values)}")

# viz
df_umap["cluster"] = [str(elt) for elt in clustering.labels_]
df_umap["noise"] = [int(elt==-1) for elt in clustering.labels_]
fig = px.scatter(
    df_umap,
    x="umap_0",
    y="umap_1",
    hover_data=[
        "title",
        # "ID",
    ],
    color="cluster",
    symbol="noise",
    category_orders={"cluster": list(np.sort(pd.unique(clustering.labels_)).astype(str))},
)
fig.show()